## WhatsApp messages bulk sending
 
* **Autor:** Braulio Arteaga  
* **Fecha de creación:** 28-01-22  
* **Fecha de actualización:**  31-01-22

Este notebook tomo como referencia principal el repositorio [whatsapp-bulk-messages-without-saving-contacts](https://github.com/inforkgodara/whatsapp-bulk-messages-without-saving-contacts) de [Ramesh Kumar](https://github.com/inforkgodara).  

¡Gracias por tu contribución!

### 1. Requisitos: 


* Descargar el chromeDriver.exe ([click aquí](https://chromedriver.chromium.org/downloads)) según la versión de tu navegador chrome. 
* Instalar los siguinetes paquetes:  
`pip install pandas`  (data manipulation)  
`pip install selenium`  (scraping)    
`pip install webdriver_manager`  (scraping)  
`pip install datetime` (current date)  
`pip install pytz` (time zone)

### 2. Código:

In [20]:
# Importar packages
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import time
from time import sleep
import pytz
from datetime import datetime

In [21]:
# Directorio actual de trabajo
import os
os.getcwd()

'C:\\Users\\braul\\Desktop\\05_personal_projects\\01_GITHUB\\WhatsApp-messages-bulk-sending\\02_codes'

In [22]:
# Estableciento rutas relativas de trabajo
MAIN = "C:/Users/braul/Desktop/05_personal_projects/01_GITHUB/WhatsApp-messages-bulk-sending"
INPUT = MAIN + "/01_input"
OUTPUT = MAIN + "/03_output"

In [23]:
# Importación de la hoja excel
excel_data = pd.read_excel(INPUT +'/list_phone_sms.xlsx', sheet_name='01_list')
data = pd.DataFrame(excel_data, columns = ['phone','message'])
data

,phone,message
0,9XXXXXXXX,¡Hola Juan! El siguiente mensaje es para recor...
1,9XXXXXXXY,¡Hola! Pedro! El siguiente mensaje es para rec...
2,9XXXXXXXZ,¡Hola Jorge! El siguiente mensaje es para reco...
3,9XXXXXXXW,¡Hola Julieta! El siguiente mensaje es para re...
4,9XXXXXXXV,¡Hola Antonella! El siguiente mensaje es para ...


In [24]:
# Añadir el código de país (Ejemplo: 51 = Perú):
data['phone'] = data['phone'].astype(str)
data["phone_cc"] = "51" + data["phone"] 
data

,phone,message,phone_cc
0,9XXXXXXXX,¡Hola Juan! El siguiente mensaje es para recor...,519XXXXXXXX
1,9XXXXXXXY,¡Hola! Pedro! El siguiente mensaje es para rec...,519XXXXXXXY
2,9XXXXXXXZ,¡Hola Jorge! El siguiente mensaje es para reco...,519XXXXXXXZ
3,9XXXXXXXW,¡Hola Julieta! El siguiente mensaje es para re...,519XXXXXXXW
4,9XXXXXXXV,¡Hola Antonella! El siguiente mensaje es para ...,519XXXXXXXV


In [29]:
# Crear la estructura del reporte final:
report_send = pd.DataFrame(columns = ['phone','status', 'motive'], index = list(range(len(data))))
report_send

,phone,status,motive
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [15]:
############
# ENVÍO DE SMS DE WSP
############

## PASO 0: Iniciar sesión en tu WhatsApp Web 
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://web.whatsapp.com')
input("Presione ENTER luego de iniciar sesión en Whatsapp Web y así dar inicio al envío de mensajes\n")


start = time.time()


for i in range(len(data)):
    try:
## PASO 1: Abrir el url correspondiente al wsp del individuo 
        report_send['phone'][i] = data['phone_cc'][i] #reporte 
        url = 'https://web.whatsapp.com/send?phone=' + str(data['phone_cc'][i]) + '&text=' + data['message'][i]
        sent = False
        driver.get(url)
## PASO 2: Encontrar el elemento clickeable necesario (HTML) para enviar el sms predefinido
        try:
            click_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, '_4sWnG')))
        except Exception as e:
            status1 = 'MENSAJE NO ENVIADO A: ' + str(data['phone_cc'][i]) 
            motive1 = 'Número sin WhatsApp o problemas de internet'
            print('\n' + status1 + '. Motivo: ' + motive1 + '\n')
            report_send['status'][i] = status1 #reporte 
            report_send['motive'][i] = motive1 #reporte 
        else:
## PASO 3: Click al elemento HTML identificado para enviar el sms predefinido
            sleep(2)
            click_btn.click()
            sent = True
            sleep(5)
            status2 = 'MENSAJE ENVIADO A: ' + str(data['phone_cc'][i])
            print(status2)
            report_send['status'][i] = status2 #reporte
    except Exception as e:
        status3 = "MENSAJE NO ENVIADO A: " + str(data['phone_cc'][i])  
        motive2 = str(e)[9:34]
        print( '\n' + status3 + '. Motivo: '+ motive2 + '\n')
        report_send['status'][i] = status3 #reporte
        report_send['motive'][i] = motive2 #reporte
        
driver.quit() #Cerrar driver luego de finalizado el loop
print("\nEl notebook terminó de ejecutarse.")


end = time.time() #Time of loop
print(f"\nLa ejecución completa del notebook demoró {round((end-start)/60,2)} minutos") #Tiempo de duración del loop



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\braul\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-15-777379362ff9>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Press ENTER after login into Whatsapp Web and your chats are visiable

Message sent to: 51933368853
Message sent to: 51922848812

Failed to send message to 51962607427. The Motive was: element click intercepted: Element <button class="_4sWnG">...</button> is not clickable at point (1000, 642)


Sorry message could not sent to 51997668202

Message sent to: 51921765439
Message sent to: 51977200827

The script executed successfully.

The total messages sending took 1.5 minutes


In [ ]:
# Obtener fecha actual en Perú
country_time = datetime.now(pytz.timezone('America/Lima')) #para averiguar tu zona horaria: print(pytz.all_timezones)
date = country_time.strftime("%d-%m-%y")

In [58]:
# Exportar reporte final
report_send.to_excel(OUTPUT + '/report_send_wsp_' + date + ".xlsx", index=False)